In [9]:
using DelimitedFiles, GLPK, JuMP

data = readdlm("ch6_ex2.csv", ',')

4×5 Matrix{Any}:
   ""  ""     25       15       10
   ""  ""       "des"    "can"    "st"
 30    "jef"  14        9        7
 20    "oma"   8       10        5

In [10]:
s_nodes = data[3:end, 2]
s = data[3:end,1]

2-element Vector{Any}:
 30
 20

In [11]:
d_nodes = data[2, 3:end]
d = data[1,3:end]

3-element Vector{Any}:
 25
 15
 10

In [12]:
s_dict = Dict(s_nodes .=>s)
d_dict = Dict(d_nodes .=>d)

Dict{SubString{String}, Int64} with 3 entries:
  "st"  => 10
  "des" => 25
  "can" => 15

In [14]:
c = data[3:end, 3:end]

2×3 Matrix{Any}:
 14   9  7
  8  10  5

In [29]:
c_dict = Dict((s_nodes[i], d_nodes[j]) => c[i,j] for i in 1:length(s_nodes), j in 1:length(d_nodes))

Dict{Tuple{SubString{String}, SubString{String}}, Int64} with 6 entries:
  ("jef", "st")  => 7
  ("oma", "st")  => 5
  ("jef", "des") => 14
  ("jef", "can") => 9
  ("oma", "can") => 10
  ("oma", "des") => 8

In [30]:
tp = Model(GLPK.Optimizer)

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: GLPK

In [31]:
@variable(tp, 0 <= x[s_nodes, d_nodes])


2-dimensional DenseAxisArray{VariableRef,2,...} with index sets:
    Dimension 1, Any["jef", "oma"]
    Dimension 2, Any["des", "can", "st"]
And data, a 2×3 Matrix{VariableRef}:
 x[jef,des]  x[jef,can]  x[jef,st]
 x[oma,des]  x[oma,can]  x[oma,st]

In [34]:
@objective(tp,Min, sum(x[i,j]*c_dict[i,j] for i in s_nodes, j in d_nodes))

14 x[jef,des] + 9 x[jef,can] + 7 x[jef,st] + 8 x[oma,des] + 10 x[oma,can] + 5 x[oma,st]

In [39]:
for i in s_nodes
    @constraint(tp, sum(x[i,j] for j in d_nodes) == s_dict[i])
end


for j in d_nodes
    @constraint(tp, sum(x[i,j] for i in s_nodes) == d_dict[j])
end

In [40]:
print(tp)

In [51]:
JuMP.optimize!(tp)

In [52]:
x_star = JuMP.value.(x)

2-dimensional DenseAxisArray{Float64,2,...} with index sets:
    Dimension 1, Any["jef", "oma"]
    Dimension 2, Any["des", "can", "st"]
And data, a 2×3 Matrix{Float64}:
  5.0  15.0  10.0
 20.0   0.0   0.0

In [53]:
tc = sum(x_star[i,j]*c_dict[i,j] for i in s_nodes, j in d_nodes)

435.0